In [18]:
import csv
import numpy as np
import pandas as pd


In [19]:
'''
The below chunk of code, drops null values from all datasets, turns the year_of_publications col to int64,
drops all images and image col from df's, converts all col to string (excluding year_of_pub), drops null values from ratings col 
'''

# Replace 'your_file.csv' with the path to your CSV file
file_paths = ['Users.csv', 'Books.csv', 'Ratings.csv']

# Load data and drop rows with null values
userData = pd.read_csv("data/"+file_paths[0], quoting=csv.QUOTE_MINIMAL, quotechar='"').dropna()


booksData = pd.read_csv("data/"+file_paths[1], quoting=csv.QUOTE_MINIMAL, quotechar='"', 
                        converters={'Year-Of-Publication': lambda x: pd.to_numeric(x, errors='coerce')}).dropna()

# Drop specified columns
columns_to_drop = ['Image-URL-S', 'Image-URL-M', 'Image-URL-L']
booksData.drop(columns=columns_to_drop, inplace=True)
booksData['Year-Of-Publication'] = booksData['Year-Of-Publication'].astype('Int64')

# Select columns to convert to strings (excluding 'Year-Of-Publication')
columns_to_convert = [col for col in booksData.columns if col != 'Year-Of-Publication']

ratingsData = pd.read_csv("data/"+file_paths[2], quoting=csv.QUOTE_MINIMAL, quotechar='"').dropna()



In [20]:
# Display data  of all columns in booksData
print("user data")
print(50 * "=")
display(userData)
print("book data")
print(50 * "=")
display(booksData)
print("rating data")
print(50 * "=")
display(ratingsData)

user data


,User-ID,Location,Age
1,2,"stockton, california, usa",18.0
3,4,"porto, v.n.gaia, portugal",17.0
5,6,"santa monica, california, usa",61.0
9,10,"albacete, wisconsin, spain",26.0
10,11,"melbourne, victoria, australia",14.0
...,...,...,...
278848,278849,"georgetown, ontario, canada",23.0
278850,278851,"dallas, texas, usa",33.0
278851,278852,"brisbane, queensland, australia",32.0
278852,278853,"stranraer, n/a, united kingdom",17.0


book data


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company
...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm)
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press


rating data


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [21]:
# combine the datasets together, doing this to reduce confusion between datasets
combine_ratings_and_books = pd.merge(ratingsData, booksData, on="ISBN")
combined_df = pd.merge(combine_ratings_and_books, userData, on="User-ID")

#get rid of publisher and year-of-publication as these have no use in knowledge-based recommendations in our case, neither provide useful information.
combined_df = combined_df.drop("Year-Of-Publication", axis=1)
combined_df = combined_df.drop("Publisher", axis=1)

combined_df = combined_df[["User-ID", "Book-Rating", "ISBN", "Book-Title", "Book-Author", "Location", "Age"]]
display(combined_df)
#display(combined_df[combined_df['Book-Title'] == "Flesh Tones: A Novel"]) # testing to see if a book has multiple ratings

,User-ID,Book-Rating,ISBN,Book-Title,Book-Author,Location,Age
0,2313,5,034545104X,Flesh Tones: A Novel,M. J. Rose,"cincinnati, ohio, usa",23.0
1,2313,9,0812533550,Ender's Game (Ender Wiggins Saga (Paperback)),Orson Scott Card,"cincinnati, ohio, usa",23.0
2,2313,8,0679745580,In Cold Blood (Vintage International),TRUMAN CAPOTE,"cincinnati, ohio, usa",23.0
3,2313,9,0060173289,Divine Secrets of the Ya-Ya Sisterhood : A Novel,Rebecca Wells,"cincinnati, ohio, usa",23.0
4,2313,5,0385482388,The Mistress of Spices,Chitra Banerjee Divakaruni,"cincinnati, ohio, usa",23.0
...,...,...,...,...,...,...,...
753291,276442,6,2264032960,L'Apprenti du diable,Ellis Peters,"genève, genève, switzerland",62.0
753292,276442,7,2862749796,Le Huit,Katherine Neville,"genève, genève, switzerland",62.0
753293,276647,0,0553571001,Christmas With Anne and Other Holiday Stories:...,L. M. Montgomery,"arlington heights, illinois, usa",13.0
753294,276647,10,0689822294,Heaven (Coretta Scott King Author Award Winner),Angela Johnson,"arlington heights, illinois, usa",13.0


##### Now that the Data is cleaned and combined into a single dataframe keeping only useful data for the recommender, we'll go ahead and do something with it now. #####

Clean the mess below: replace with something like scipy to make a sparser matrix

Note: I verified that all users had at least 1 rating

In [22]:
''' so I kept getting the error ValueError: negative dimensions are not allowed when trying to create a user-book matrix for
    recommendations because it lists all books for all users whether they have a rating of not for the novel. To fix this temporarily
    I removed half of the data, which then it worked but not for cosine similarity (tries to get 151GB of memory to store the data)'''

# to use scipy coo_matrix, we need the isbn to be numeric, after a few attempts of converting most isbn's from objects to int I found 
# making a dictionary where the index n is the key was a better approach to ensure our data is untouched.
# *none is removed for being non-numeric, all ISBN's can be abstracted to a key (i)*
ISBN_dictionary = {ISBN: i for i, ISBN in enumerate(combined_df['ISBN'].unique())}

combined_df['ISBN-Key'] = combined_df["ISBN"].map(ISBN_dictionary)

display(combined_df)

,User-ID,Book-Rating,ISBN,Book-Title,Book-Author,Location,Age,ISBN-Key
0,2313,5,034545104X,Flesh Tones: A Novel,M. J. Rose,"cincinnati, ohio, usa",23.0,0
1,2313,9,0812533550,Ender's Game (Ender Wiggins Saga (Paperback)),Orson Scott Card,"cincinnati, ohio, usa",23.0,1
2,2313,8,0679745580,In Cold Blood (Vintage International),TRUMAN CAPOTE,"cincinnati, ohio, usa",23.0,2
3,2313,9,0060173289,Divine Secrets of the Ya-Ya Sisterhood : A Novel,Rebecca Wells,"cincinnati, ohio, usa",23.0,3
4,2313,5,0385482388,The Mistress of Spices,Chitra Banerjee Divakaruni,"cincinnati, ohio, usa",23.0,4
...,...,...,...,...,...,...,...,...
753291,276442,6,2264032960,L'Apprenti du diable,Ellis Peters,"genève, genève, switzerland",62.0,228991
753292,276442,7,2862749796,Le Huit,Katherine Neville,"genève, genève, switzerland",62.0,228992
753293,276647,0,0553571001,Christmas With Anne and Other Holiday Stories:...,L. M. Montgomery,"arlington heights, illinois, usa",13.0,228993
753294,276647,10,0689822294,Heaven (Coretta Scott King Author Award Winner),Angela Johnson,"arlington heights, illinois, usa",13.0,228994


In [23]:
import scipy


# From the combined_df create the user to book matrix that will be used in the recommender
# The row/index is the user, the columns are the ISBN of the book and its rating
book_to_user_rating_matrix = scipy.sparse.coo_matrix((combined_df["Book-Rating"], (combined_df["User-ID"], combined_df["ISBN-Key"])))

display(book_to_user_rating_matrix)

<278853x228996 sparse matrix of type '<class 'numpy.int64'>'
	with 753296 stored elements in COOrdinate format>

In [24]:
import sklearn.metrics as skm

# get the cosine similarity between all of the books and their respective user-ratings
books_most_similar_ratings = skm.pairwise.cosine_similarity(book_to_user_rating_matrix.T, dense_output=False) #False = scipy sparse object, true = numpy

display(books_most_similar_ratings)

<228996x228996 sparse matrix of type '<class 'numpy.float64'>'
	with 59143112 stored elements in Compressed Sparse Row format>

In [58]:

def knowledge_based_recommender(combined_df, user_id, book_to_user_rating_matrix, books_most_similar_ratings, ISBN_dictionary):
    
    #get the similarities between users for the current user, ensure that the respective user data is removed from the recommendations
    user_to_user_similarities = skm.pairwise.cosine_similarity(book_to_user_rating_matrix.getrow(user_id), book_to_user_rating_matrix).ravel() #ravel flattens the array to 1D
    user_to_user_similarities[user_id] = 1

    
    #get the most similar users to the current user, done with argsort to get the sorted order, limiting to top 5 users
    most_similar_users = user_to_user_similarities.argsort()[::-1][:5]
    
    #get the ISBN's from these users, map the ISBN keys back to the corresponding ISBN
    #most_similar_users_isbn_keys = user_to_user_similarities.index[most_similar_users] <== previous thing
    #Get ISBN column from filtered combined_df, filter by User-ID
    most_similar_users_isbn_keys = combined_df.loc[combined_df['User-ID'].isin(most_similar_users)]['ISBN']
    
    #redundant because it just returns the ISBN-key values which arent needed to get the books from bookData
    most_similar_users_isbns = [ISBN_dictionary[i] for i in most_similar_users_isbn_keys]# if i >= 0]

    #return the recommended books that were found back to the user

    #filters by ISBN series above, gets the book title specifically
    books_recommended = booksData[booksData['ISBN'].isin(most_similar_users_isbn_keys)]['Book-Title']


    return books_recommended

test_user_id = 35953 

test_output = knowledge_based_recommender(combined_df, test_user_id, book_to_user_rating_matrix, books_most_similar_ratings, ISBN_dictionary)

print(f"books for {test_user_id} are:\n{test_output}")


books for 35953 are:
491                      The First Wives Club Movie Tie In
2241                                  The Andromeda Strain
3896                                       Midnight Voices
4234                                          The Alienist
4578                       Angela's Ashes (MMP) : A Memoir
5005                                         Death du Jour
7383                   Back When We Were Grownups: A Novel
7710     Grave Secrets (Temperance Brennan Novel (Hardc...
13524                                                Roots
16453    Stop the Insanity! Eat, Breathe, Move, Change ...
23784                                    Legacy of Silence
27704                         The Last Time I Wore a Dress
28946    Inner Simplicity : 100 Ways to Regain Peace an...
36224    The Return of the Indian (Indian in the Cupboard)
46900                      Steel Gauntlet (Starfist, No 3)
46919    Starfist: Hangfire : Book VI (Starfist (Paperb...
47059                              